In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy
import os

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Bidirectional

In [2]:
# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(0)

In [3]:
# 학습셋, 테스트셋 지정하기
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = 10000)

In [4]:
# 데이터 전처리
x_train = sequence.pad_sequences(x_train, maxlen = 200)
x_test = sequence.pad_sequences(x_test, maxlen = 200)

In [5]:
# 모델의 설정
model = Sequential()
model.add(Embedding(10000, 200))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding = 'valid', activation = 'relu', strides = 1))
model.add(MaxPooling1D(pool_size = 4))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation = 'sigmoid'))

In [6]:
# 모델 요약
model.build(input_shape = (None, 100))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 200)         2000000   
                                                                 
 dropout (Dropout)           (None, None, 200)         0         
                                                                 
 conv1d (Conv1D)             (None, None, 64)          64064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 64)         0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0

In [7]:
# 모델 컴파일
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# 콜백 설정
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 4, verbose = 1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 2, verbose = 1)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor = 'val_loss', save_best_only = True, verbose = 1)

: 

In [ ]:
# 모델의 실행
history = model.fit(x_train, y_train, batch_size = 64, epochs = 20, validation_data = (x_test, y_test), callbacks = [early_stopping, reduce_lr, model_checkpoint])

Epoch 1/20


In [ ]:
# 모델의 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

In [1]:
# 그래프 그리기
history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']
accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

epochs = range(1, len(loss) + 1)

# 손실 그래프
plt.figure(figsize = (12, 6))
plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 정확도 그래프
plt.figure(figsize = (12, 6))
plt.plot(epochs, accuracy, 'ro', label = 'Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined